# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra.cluster import Cluster

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# # Get event data path
filepath = os.getcwd() + '/event_data'

# create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Define functions for pre-processing

In [3]:

full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [7]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [9]:

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifyks 
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)


session.set_keyspace('sparkifyks')

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

# Based on the above requirements, we could design the data model as follows:

##### Data Modeling
>1.The combination of sessionId and itemInSession is unique, and we need results based on sessionId and itmeInSession, so these two columns could be the Composite Primary Key of the table. 
 >>2.Since itemInSession could be skewed to some specific value, it's better to use sessionId as Priamary key so the data could be evenly distributed on the nodes
     >>> 3.We need the artist name, song title and song length based on the sessionId and itemInSession
Based on the above requirements, we could design the data model as follows:


#### Create the table sessions

In [11]:

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_session_by_item (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)
                    

##### Insert data into the table sessions

In [12]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO songinfo_by_session_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
      

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Run the test query
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, length
    FROM songinfo_by_session_by_item
    WHERE
        sessionId=338 AND
        itemInSession=4;
""")

df = result._current_rows 
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2:Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
## Data Modeling

### 1.Since we need results based on userid and sessionid, it could make good perfomance to use userid and sessionid as partition keyto make sure the sessions belonging to the same user could be distributed to same node 2. Since the data needs to be sorted by itemInSession, the itemInSession should be the clustering key 3. Besides Primary Key, we need artist name, song title and user name for this query Based on the above requirements, we could design the data model as follows:

###  COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Create the table song_playlist_session

In [14]:

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_user_by_session (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)
except Exception as e:
    print(e)
                    

Insert data into the table song_playlist_session

In [15]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

Run the test query

In [16]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM songinfo_by_user_by_session
    WHERE
        userId=10 AND
        sessionId=182
    ORDER BY itemInSession;
""")

df = result._current_rows 
df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz



 In sessionId=182, userId=10 Sylvie Cruz listened to 4 songs: 'Keep on Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.

## Query 3:
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Data Modeling
> 1. Since we need the results based on song name, firstName and lastName might be not unique, we need to use song name and userid as Primary Key
> 2. Use the song name as partition key so the users data listened to the same song could be distributed on the same node, it    will improve the query performance
>3. we need firatName and lastName as well for this query
    Based on the above requirements, we could design the data model as follows:

 ### Insert data into the table users_playlist

In [17]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS userinfo_by_song (
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            userId INT,
            PRIMARY KEY (song, firstName, lastName, userId));
    """)
except Exception as e:
    print(e)

### Insert data into the table users_playlist

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO userinfo_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

#### Run the test query 

In [19]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT firstName, lastName
    FROM userinfo_by_song
    WHERE song='All Hands Against His Own';
""")

df = result._current_rows 
df

,firstname,lastname
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


Three users listened to the song 'All Hands Against His Own': Jacqueline Lynch, Sara Johnson, and Tegan Levine.

### Drop the tables before closing out the sessions

In [21]:
try:
    session.execute("DROP TABLE songinfo_by_session_by_item")
    session.execute("DROP TABLE songinfo_by_user_by_session")
    session.execute("DROP TABLE userinfo_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()